In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
def scraping_utmb():
    url = 'https://utmb.world/utmb-index/runner-search'
    browsers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \(KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36'}
    page = requests.get(url, headers=browsers)

    soup = BeautifulSoup(page.content, 'html.parser')
    runners = soup.select('.my-table_row__nlm_j')

    dict_names = {
        i.find('a', attrs={'class': 'link_link__96ppl link-underline'}).text: [i.find('div', attrs={'class': 'my-table_cell__z__zN col-2'}).text,
        str(i.find('div', attrs={'data-label': "nationality"}))[180:182],
        i.find('div', attrs={'data-label': "Gender"}).text,
        i.find('div', attrs={'data-label': "Age Group"}).text, 
        'https://utmb.world/'+ str(i.find('a', attrs={'class': 'link_link__96ppl link-underline'})['href'])
        ]
        for i in runners
    }

    return dict_names

resultado = scraping_utmb()

df = pd.DataFrame(resultado.values(), index=resultado.keys())
df = df.set_axis(['Posição', 'Nacionalidade', 'Gênero', 'Faixa Etária', 'Site'], axis=1)
df


,Posição,Nacionalidade,Gênero,Faixa Etária,Site
Remi BONNET,943,ch,Men,20-34,https://utmb.world//en/runner/687094.remi.bonnet
Kilian JORNET BURGADA,941,es,Men,35-39,https://utmb.world//en/runner/2704.kilian.jorn...
Patrick KIPNGENO,941,ke,Men,20-34,https://utmb.world//en/runner/4595246.patrick....
Jonathan ALBON,938,gb,Men,35-39,https://utmb.world//en/runner/527631.jonathan....
Jim WALMSLEY,936,us,Men,20-34,https://utmb.world//en/runner/485740.jim.walmsley
Philemon KIRIAGO,933,ke,Men,20-34,https://utmb.world//en/runner/4595248.philemon...
Stian ANGERMUND,932,no,Men,35-39,https://utmb.world//en/runner/493528.stian.ang...
Adam PETERMAN,932,us,Men,20-34,https://utmb.world//en/runner/3572577.adam.pet...
Petter ENGDAHL,927,se,Men,20-34,https://utmb.world//en/runner/1065223.petter.e...
Elhousine ELAZZAOUI,925,ma,Men,20-34,https://utmb.world//en/runner/2161523.elhousin...


---
Selenium
---

In [11]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By
from time import sleep

In [48]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get(url='https://utmb.world/utmb-index/runner-search')

accept = driver.find_element(by=By.ID, value="onetrust-accept-btn-handler")
accept.click()
next = driver.find_elements(by=By.TAG_NAME, value="a")
# next[98].click()
next[98].get_attribute('aria-label')

'Next page'